In [1]:
from authenticate import authenticate_google_calendar 

In [2]:
import datetime

def fetch_study_events(service, calendar_id="primary"):
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    # Calculate the start of the month
    start_of_month = (
        datetime.datetime.utcnow().replace(day=1) - datetime.timedelta(days=1)
    ).replace(day=1).isoformat() + "Z"
    print("Fetching past month events")
    events_result = (
        service.events()
        .list(
            calendarId=calendar_id,
            timeMin=start_of_month,
            timeMax=now,
            maxResults=100,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
        print("No past month events found.")
        return []

    for event in events:
        start = event["start"].get("dateTime", event["start"].get("date"))
        print(start, event["summary"])

    return events


def study_stats(events):
    total_study_time = 0
    for event in events:
        start = datetime.datetime.fromisoformat(
            event["start"].get("dateTime", event["start"].get("date"))
        )
        end = datetime.datetime.fromisoformat(
            event["end"].get("dateTime", event["end"].get("date"))
        )
        duration = (end - start).total_seconds() / 3600  # Convert to hours
        total_study_time += duration
    print(f"Total study time: {total_study_time} hours")


def create_study_plan(service, subjects, calendar_id="primary"):
    
    def delete_overlapping_events(start_time, end_time):
        """Deletes events in the study plan that overlap with the given time range."""
        events_result = service.events().list(
            calendarId=calendar_id, 
            timeMin=start_time.isoformat(),
            timeMax=end_time.isoformat(), 
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])

        for event in events:
            if 'summary' in event and event['summary'] in subjects:
                service.events().delete(calendarId=calendar_id, eventId=event['id']).execute()
                print(f"Deleted overlapping event: {event['summary']} from {event['start']['dateTime']} to {event['end']['dateTime']}.")

    now = datetime.datetime.now(datetime.timezone.utc)
    current_date = now.date()
    study_dates = [current_date, current_date + datetime.timedelta(days=1)]
    total_weight = sum(subjects.values())

    for study_date in study_dates:
        if study_date == current_date and now.time() > datetime.time(9, 0):
            start_time = max(now, datetime.datetime.combine(study_date, datetime.time(9, 0), tzinfo=datetime.timezone.utc))
        else:
            start_time = datetime.datetime.combine(study_date, datetime.time(9, 0), tzinfo=datetime.timezone.utc)

        end_time = datetime.datetime.combine(study_date, datetime.time(17, 0), tzinfo=datetime.timezone.utc)
        lunch_start = datetime.datetime.combine(study_date, datetime.time(12, 0), tzinfo=datetime.timezone.utc)
        lunch_end = lunch_start + datetime.timedelta(hours=1)

        # Delete overlapping events in the study plan
        delete_overlapping_events(start_time, end_time)

        while start_time < end_time:
            if lunch_start <= start_time < lunch_end:
                start_time = lunch_end  # Skip lunch hour
                continue

            for subject, weight in subjects.items():
                study_duration = datetime.timedelta(minutes=40 * weight / total_weight)
                gap_duration = datetime.timedelta(minutes=5)

                event_end = start_time + study_duration
                if event_end > end_time or (lunch_start <= event_end < lunch_end):
                    break  # Skip to the next subject if the event would end outside of study hours or during lunch

                event = {
                    "summary": subject,
                    "start": {
                        "dateTime": start_time.isoformat(),
                        "timeZone": "UTC",
                    },
                    "end": {
                        "dateTime": event_end.isoformat(),
                        "timeZone": "UTC",
                    },
                }
                service.events().insert(calendarId=calendar_id, body=event).execute()
                print(f"Added {subject} study block from {start_time} to {event_end}.")

                start_time = event_end + gap_duration  # Next event starts after a gap

            if start_time >= end_time:
                break  # If the day's study time is over, exit the loop

In [3]:
calendar_id = "0ca09266015f691eebe0d00c6f3ed7a784713e0160a694b8f7929add00cb1aa1@group.calendar.google.com"

In [4]:

service = authenticate_google_calendar()
events = fetch_study_events(service, calendar_id)  # Specify your calendar ID for "studying"
study_stats(events)

subjects = {'Maths': 3, 'Physics': 2, 'Computer Science': 1}  # Example subjects with weights
create_study_plan(service, subjects, calendar_id)  # Specify your calendar ID


Fetching past month events
2024-03-02T09:30:00Z epq
2024-03-03T09:00:00Z epq
2024-03-03T09:45:00Z end of week revision maths
2024-03-03T10:15:00Z end of week revision physics
2024-03-03T10:45:00Z end of week revision comp sci
2024-03-05T16:00:00Z epq
2024-03-12T16:00:00Z epq
2024-03-14T10:30:00Z epq
2024-03-26T16:00:00Z epq
2024-04-02T13:00:00Z maths
2024-04-03T09:00:00Z Maths
2024-04-03T09:25:00Z Maths
2024-04-03T09:50:00Z Maths
2024-04-03T10:15:00Z Maths
2024-04-03T10:40:00Z Maths
2024-04-03T11:05:00Z Maths
2024-04-03T11:30:00Z Maths
2024-04-03T12:45:00Z maths
2024-04-03T14:44:03Z Maths
Total study time: 10.916666666666668 hours
Deleted overlapping event: Maths from 2024-04-03T14:44:03Z to 2024-04-03T15:04:03Z.
Deleted overlapping event: Physics from 2024-04-03T15:09:03Z to 2024-04-03T15:22:23Z.
Deleted overlapping event: Computer Science from 2024-04-03T15:27:23Z to 2024-04-03T15:34:03Z.
Added Maths study block from 2024-04-03 14:50:43.507582+00:00 to 2024-04-03 15:10:43.507582+00:0

KeyboardInterrupt: 